# MSDS 631 - Supplemental Notes
## Reshaping and Filtering Data Example

Now that we've learned shortcuts for opening data into DataFrames, it seems we should always be able to just automatically load data without having to use any base Python to parse data. However, there are still countless situations where you are not creating DataFrames until after you've reorganized and reshaped your data. One example is having data that is so big that it requires us to remove some data before trying to create our DataFrame (else run into computer memory issues).

Our objective in this example is to take Yelp review data and remove the actual review text while putting the other data into the order we want.

Let's start by opening a tiny subset of the data we want to work with.

In [97]:
import json
import pandas as pd

In [98]:
#Due to issues with the raw data file, I had to do some tricky things to get the data.
#DO NOT worry about understanding what is in this cell
def review_generator():
    with open('data/reviews_sample.json', 'r') as f:
        reviews = f.read().strip().split('\n')
    for review in reviews:
        yield json.loads(review)
        
def get_all_review_data():
    reviews = []
    for review in review_generator():
        reviews.append(review)
    return reviews

In [99]:
reviews = get_all_review_data()

In [100]:
#Let's take a look at the first review
reviews[0]

{'business_id': 'ujmEBvifdJM6h6RLv4wQIg',
 'cool': 0,
 'date': '2013-05-07 04:34:36',
 'funny': 1,
 'review_id': 'Q1sbwvVQXV2734tPgoKj4Q',
 'stars': 1.0,
 'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.',
 'useful': 6,
 'user_id': 'hG7b0MtEbXx5QzbzE6C_VA'}

We have 9 keys:
- business_id
- cool
- date (as a string)
- funny
- review_id
- stars
- text
- useful
- user_id

#### Creating Ordered List from Data in Dictionary

What we want is to take all of these reviews and create a list of lists that contain only the data we want and in the order we want it.

To take a single review dictionary and convert it into a single ordered list, we create a function.

In [101]:
def parse_review(review):
    biz_id = review['business_id']
    cool_count = review['cool']
    review_date = review['date']
    funny_count = review['funny']
    review_id = review['review_id']
    stars = review['stars']
    useful = review['useful']
    user_id = review['user_id']
    my_list = [user_id, review_id, review_date, biz_id, stars, cool_count, funny_count, useful]
    return my_list

In [102]:
#A second way we could have defined the parsing function as as follows:
def parse_review_alternative(review):
    key_order = ['user_id', 'review_id', 'date', 'business_id', 'stars', 'cool', 'funny', 'useful']
    my_list = []
    for key in key_order:
        my_list.append(review[key])
    return my_list

In [103]:
parse_review(reviews[0])

['hG7b0MtEbXx5QzbzE6C_VA',
 'Q1sbwvVQXV2734tPgoKj4Q',
 '2013-05-07 04:34:36',
 'ujmEBvifdJM6h6RLv4wQIg',
 1.0,
 0,
 1,
 6]

In [104]:
#Double check to see if the two parsing methods are equivalent
parse_review(reviews[0]) == parse_review_alternative(reviews[0])

True

#### Parsing all reviews
Now we want to create a list of lists, where each internal list is simply the parsed values from individual reviews. Afterwards we can turn that list of lists into a DataFrame.

In [105]:
all_parsed_reviews = []
for review in reviews: #reviews is a list of dictionaries, and review is a dictionary of a single review
    parsed_review = parse_review(review)
    all_parsed_reviews.append(parsed_review)

reviews_df = pd.DataFrame(all_parsed_reviews)
reviews_df.head()

,0,1,2,3,4,5,6,7
0,hG7b0MtEbXx5QzbzE6C_VA,Q1sbwvVQXV2734tPgoKj4Q,2013-05-07 04:34:36,ujmEBvifdJM6h6RLv4wQIg,1.0,0,1,6
1,yXQM5uF2jS6es16SJzNHfg,GJXCdrto3ASJOqKeVWPi6Q,2017-01-14 21:30:33,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0
2,n6-Gk65cPZL6Uz8qRm3NYw,2TzJjDVDEuAW6MR5Vuc1ug,2016-11-09 20:09:03,WTqjgwHlXbSFevF32_DJVw,5.0,0,0,3
3,dacAIZ6fTM6mqwW5uxkskg,yi0R0Ugj_xUx_Nek0-_Qig,2018-01-09 20:56:38,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0
4,ssoyf2_x0EQMed6fgHeMyQ,11a8sVPMUFtaC7_ABRkmtw,2018-01-30 23:07:38,b1b1eb3uo-w561D0ZfCEiQ,1.0,0,0,7


In [106]:
#Let's add our column names now
reviews_df.columns = ['user_id', 'review_id', 'review_date', 'business_id', 'star_rating', 'cool_count',
                      'funny_count', 'useful_count']
reviews_df.head()

,user_id,review_id,review_date,business_id,star_rating,cool_count,funny_count,useful_count
0,hG7b0MtEbXx5QzbzE6C_VA,Q1sbwvVQXV2734tPgoKj4Q,2013-05-07 04:34:36,ujmEBvifdJM6h6RLv4wQIg,1.0,0,1,6
1,yXQM5uF2jS6es16SJzNHfg,GJXCdrto3ASJOqKeVWPi6Q,2017-01-14 21:30:33,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0
2,n6-Gk65cPZL6Uz8qRm3NYw,2TzJjDVDEuAW6MR5Vuc1ug,2016-11-09 20:09:03,WTqjgwHlXbSFevF32_DJVw,5.0,0,0,3
3,dacAIZ6fTM6mqwW5uxkskg,yi0R0Ugj_xUx_Nek0-_Qig,2018-01-09 20:56:38,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0
4,ssoyf2_x0EQMed6fgHeMyQ,11a8sVPMUFtaC7_ABRkmtw,2018-01-30 23:07:38,b1b1eb3uo-w561D0ZfCEiQ,1.0,0,0,7


#### Storing parsed reviews (i.e ordered list of values) in dictionaries based on star rating

Now, instead of creating one master DataFrame, we want to create five DataFrames based on star rating. To do this we can build our list of lists, but each outer list is put into a dictionary keyed on the star rating.

In [107]:
all_reviews_stars = []
for review in reviews:
    all_reviews_stars.append(review['stars'])
all_available_stars = set(all_reviews_stars)
all_available_stars

{1.0, 2.0, 3.0, 4.0, 5.0}

Note that these stars are floats and we prefer them as integers, so we'll convert them before adding them to the dictionary keys.

In [108]:
star_reviews = {}
for i in all_available_stars:
    rounded_stars = int(i)
    star_reviews[rounded_stars] = []
star_reviews

{1: [], 2: [], 3: [], 4: [], 5: []}

Now let's go through the reviews and put the reviews where we want them.

In [109]:
for review in reviews:
    parsed_review = parse_review(review)
    star_rating = review['stars']
    star_reviews[star_rating].append(parsed_review) 

Let's see how many ratings wound up in each key.

In [110]:
for key in star_reviews:
    print(key, len(star_reviews[key]))

1 155
2 91
3 109
4 235
5 410


Let's go ahead and convert those list of lists into DataFrames now.

In [112]:
for key in star_reviews:
    star_df = pd.DataFrame(star_reviews[key])
    star_df.columns = ['user_id', 'review_id', 'review_date', 'business_id', 'star_rating', 'cool_count',
                      'funny_count', 'useful_count']
    star_reviews[key] = star_df

#### Another approach - much less efficient
To create our star_reviews data, here's another much less efficient way to solve our problem.

In [113]:
star_reviews = {} #Initialize dictionary
for i in range(1,6): #Assume the stars will always be between 1 and 5
    star_reviews[i] = [] #Initialize an empty list as a value for our star keys
    for review in reviews: #Loop through all of the reviews
        if i == review['stars']: #Check to see if the review has the same star value as the key we just added
            star_reviews[i].append(parse_review(review)) #If so, add the parsed data

The problem with the solution above is that it forces us to look through the reviews data five times instead of just once. This may not be a big deal with tiny amounts of data like our sample, but the full review data set has 6.7 million reviews. That means that doing that 5 times means we have over 33 million loops that we're running. That's 27 million wasted loops and many more than that computations.

### Storing reviews by year

Instead of using star ratings as keys, let's put the data into keys by the year of the review.

In [122]:
all_reviews_years = []
for review in reviews:
    year = review['date'][:4] #Date is a string, and we just want the first 4 characters
    all_reviews_years.append(year)
all_available_years = set(all_reviews_years)
all_available_years

{'2004',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018'}

In [123]:
#Since our years are strings and we prefer ints, we'll convert them before adding the keys
year_reviews = {}
for year in all_available_years:
    converted_year = int(year)
    year_reviews[converted_year] = []
year_reviews

{2004: [],
 2006: [],
 2007: [],
 2008: [],
 2009: [],
 2010: [],
 2011: [],
 2012: [],
 2013: [],
 2014: [],
 2015: [],
 2016: [],
 2017: [],
 2018: []}

In [124]:
#Let's add the parsed data now
for review in reviews:
    parsed_review = parse_review(review)
    year = int(review['date'][:4])
    year_reviews[year].append(parsed_review)

In [125]:
#Turn each list of lists into a DataFrame
for year in year_reviews:
    year_df = pd.DataFrame(year_reviews[year])
    year_df.columns = ['user_id', 'review_id', 'review_date', 'business_id', 'star_rating', 'cool_count',
                      'funny_count', 'useful_count']
    year_reviews[year] = year_df    

In [126]:
year_reviews.keys()

dict_keys([2016, 2017, 2018, 2004, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [127]:
#Let's take a look at the first few reviews from 2014 DataFrame
year_reviews[2014].head()

,user_id,review_id,review_date,business_id,star_rating,cool_count,funny_count,useful_count
0,_N7Ndn29bpll_961oPeEfw,4bUyL7lzoWzDZaJETAKREg,2014-06-27 21:19:23,y-Iw6dZflNix4BdwIyTNGA,3.0,0,0,0
1,vDr8hBNYigiY70fk1kF5iQ,W1wbNaUnbMy4b9QqY3_SAg,2014-08-10 22:07:35,hd5xm20tfSa70-6UqD9-bg,1.0,1,1,7
2,q3GeSW9dWN9r_ocqFkhrvg,1daGBpea0sleayFeeXuoYw,2014-04-19 15:03:17,9nTF596jDvBBia2EXXiOOg,1.0,0,0,1
3,C_hUvw2z0R-Rv0yZb6QCZA,0AsmPiAQduxh5jE_si8cLA,2014-12-17 19:04:33,dm6sO_Y8JdKTE1ZM955yug,5.0,0,0,0
4,1fi6x4tnJtlVWaJmoIO9XA,bjD0Dqn3k-fi00BXatrytg,2014-06-27 21:32:31,XTzKRvWciP_BZ9imk8mmPQ,1.0,0,1,5
